In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import os
import torch

In [ ]:
X=pd.read_csv('/content/drive/MyDrive/bigdata_pj-20221212T104521Z-001/bigdata_pj/all_poi.csv')

In [ ]:
X.columns

Index(['ICAO', 'latitude', 'longitude', 'name', 'Enplanements', 'natural',
       'amenity', 'shop', 'railway', 'public_transport', 'tourism', 'highway'],
      dtype='object')

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X['natural	amenity	shop	railway	public_transport tourism highway'.split()], X['Enplanements'],random_state=1)

PCA, if necessary

In [ ]:
pca=PCA(n_components=0.95)
pca.fit(X_train)

PCA(n_components=0.95)

In [ ]:
X_train_2=pca.fit_transform(X_train)

In [ ]:
X_train

,natural,amenity,shop,railway,public_transport,tourism,highway
112,112,65,29,0,4,23,0
69,14,460,358,48,36,74,1
14,1,1646,464,118,5298,83,3265
166,135,32,14,0,0,3,0
181,5,7,3,0,0,2,0
...,...,...,...,...,...,...,...
203,12,1,3,0,1,5,0
137,1,97,31,3,9,3,9
72,53,327,119,0,1,65,7
140,0,407,103,12,9,71,7


MLP

In [ ]:
regr = MLPRegressor(hidden_layer_sizes=(7,20,100,100,50,20,10),solver='lbfgs',alpha=0.01,
                    learning_rate='adaptive',learning_rate_init=0.001).fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [ ]:
regr.score(X_test,y_test)

-0.9115605129905848

Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
regr2 = Ridge(alpha=0.1)
regr2.fit(X_train,y_train)
regr2.score(X_test,y_test)

0.08074684004287247

In [ ]:
from sklearn.svm import LinearSVR

In [ ]:
regr3 = LinearSVR()
regr3.fit(X_train,y_train)
regr3.score(X_test,y_test)

-0.015948201687853825

In [ ]:
import torch
import torch.nn as nn
def df_to_tensor(x):
    return torch.tensor(x.values)

In [ ]:
X_train_tensor=df_to_tensor(X_train)
X_test_tensor=df_to_tensor(X_test)
y_train_tensor=df_to_tensor(y_train).unsqueeze(-1)
y_test_tensor=df_to_tensor(y_test).unsqueeze(-1)

In [ ]:
y_train_tensor.unsqueeze(-1)

In [ ]:
n_input, n_hidden, n_out = 7,[5,5,5,5],1
model = nn.Sequential(nn.Linear(n_input, n_hidden[0]),
                      nn.Sigmoid(),
                      nn.Linear(n_hidden[0], n_hidden[1]),
                      nn.Sigmoid(),
                      nn.Linear(n_hidden[1], n_hidden[2]),
                      nn.Sigmoid(),
                      nn.Linear(n_hidden[2], n_hidden[3]),
                      nn.Sigmoid(),
                      nn.Linear(n_hidden[3], n_out),
                      nn.Sigmoid())

In [ ]:
learning_rate=[1e-5,1e-4,1e-3,1e-2,1e-1,1]
for lr in learning_rate:
    loss_function = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    losses = []
    for epoch in range(5000):
        # print(list(model.parameters()))
        pred_y = model(X_train_tensor.float())
        loss = loss_function(pred_y, y_train_tensor.float())
        losses.append(loss.item())
        model.zero_grad()
        with model.no_grad():
            
        loss.backward()
        optimizer.step()
    print(loss.item(), end=' ')

IndentationError: ignored